<img src="images/Biopython_logo.png" alt="Logo Biopython" style="width: 300px;"/>

# Biopython básico

El proyecto Biopython mantiene una librería para el manejo de datos biológicos con Python creada por una asociación internacional de desarrolladores y disponible libremente en https://biopython.org/

Esta librería facilita, entre otras cosas:
- el manejo de secuencias y anotaciones
- la descarga de secuencias desde NCBI y otras bases de datos biológicas
- el trabajo con archivos estándares
- la realización de alineamientos
- la invocación a BLAST via Internet o localmente
- el manejo de información de genética poblacional, filogenética y otros
- la aplicaicón de métodos de aprendizaje supervisados
- el despliegue de diagramas 

Este tutorial explica cómo utilizar Biopython para representar secuencias, realizar consultas a NCBI e invocar a BLAST.

Existe un documento que especifica cómo trabajar con Biopython, cuya última versión puede encontrarse [acá](http://biopython.org/DIST/docs/tutorial/Tutorial.html)


## Objetos Secuencia (Seq)

Uno de los objetos más utilizados en Biopython es el de **secuencia**, el cual puede almacenar ADN, ARN, y Aminoácidos (proteínas). Para crearlos, simplemente se invoca el constructor de la clase **Seq** con el string de la secuencia:

In [ ]:
from Bio.Seq import Seq
my_seq = Seq('GATTACA')
print(type(my_seq))
print(my_seq)

La clase **Seq** implementa métodos comunmente requeridos, tales como obtención subsecuencias, obtención del complemento, cálculo de %GC, etc

In [ ]:
print(my_seq)
print(my_seq[0])
print(my_seq[0:3])
print('--------------------')
print(my_seq.complement())
print(my_seq[::-1])                   # reverse
print(my_seq.reverse_complement())
print('--------------------')
pGC = 100*(my_seq.count('G')+my_seq.count('C'))/len(my_seq)
print(f'%GC: {pGC}')

El cálculo de %GC también puede obtenerse importando el método **GC**

In [ ]:
from Bio.SeqUtils import GC
GC(my_seq)

Las secuencias pueden concatenarse como si fueran strings, y transformarse a strings de ser necesario

In [ ]:
print(my_seq[2:4])
print(my_seq[0:2])
nuevaSeq = my_seq[2:4] + my_seq[0:2] # concatena secuencias
print(type(nuevaSeq))
print(nuevaSeq)
print('---------------')
SeqStr = str(my_seq) # obtiene string desde objeto Seq
print(type(SeqStr))
print(SeqStr)

**Seq** implementa métodos para transcribir de DNA a RNA, así como para retrotranscribir desde RNA a DNA

In [ ]:
print(my_seq)
rna = my_seq.transcribe()
dna = rna.back_transcribe()
print(rna)
print(dna)

**Seq** implementa métodos para traducir de ARN (o ADN) a aminoácidos. Sólo se convertirán los codones completos. Existen varios parámetros que se pueden especificar en la traducción, los que pueden ser vistos en la documentación

In [ ]:
print(len(rna))
print(f'{rna} translate:  ',rna.translate())
print(f'{dna} translate:  ',dna.translate())
print(f'{dna[0:6]} translate:   ',dna[0:6].translate())

## Objetos de anotación de secuencias (SeqRecord)

Mientras que **Seq** almacena secuencias, la clase **SeqRecord** provee la posibilidad de anotarlas, asociando a la secuencia una serie de metadata tal como el identificador de ésta, el organismo de origen, la calidad, etc. Para crear un objeto **SeqRecord** se requiere un objeto **Seq**

In [ ]:
from Bio.Seq import Seq
seq = Seq('GATTACA')

from Bio.SeqRecord import SeqRecord
seqr = SeqRecord(seq)
seqr.id = 'AB12345'
seqr.description = 'obtenido desde PCR'
seqr

Uno de los campos de Seqrecord es **annotations**, el cual es un diccionario que puede almacenar información diversa

In [ ]:
seqr.annotations["origen"] = "Homo sapiens"
print(seqr.annotations)
seqr.annotations["phred_quality"] = [32,40,38,30,45,51,43]
print(seqr.annotations)

Los objetos SeqRecord contienen los siguientes atributos:

- **.seq** la secuencia misma, un objeto *Seq*
- **.id** el identificador primario de la secuencia
- **.name** un string con más información
- **.description** un srring con explicación de consumo humano
- **.letter_annotations** anotación letra por letra del mismo largo de la secuencia, como por ejemplo los scores de calidad
- **.annotations** diccionario con anotaciones diversas
- **.features** lista de objetos **SeqFeature** con anotación de la secuencia
- **.dbxrefs** lista de referencias cruzadas con oatras bases de datos

## Parseo con archivos (SeqIO)

La clase **SeqIO** permite el parseo (lectura+procesamiento) de archivos estándares. Estos archivos pueden existir local o remotamente, y estar comprimidos o no (ver documentación). A continuación se muestra la lectura de un archivo **FASTA**.

In [ ]:
!cat Test.fasta

In [ ]:
from Bio import SeqIO
for seq_record in SeqIO.parse('Test.fasta','fasta'):
    print(seq_record.id)           # ID
    print(seq_record.description)  # descripción (toda la fila)
    print(repr(seq_record.seq))    # secuencia
    print(len(seq_record))         # largo de secuencia
    print('-----------------------')

Otro formato parseable es un registro **GenBank**, como se muestra a continuación. Note que el cambio en el código es mínimo

In [ ]:
# (archivo largo, sólo se mostrarán las primeras 40 líneas)
!head -n 40 PKD2.gb

In [ ]:
from Bio import SeqIO
for seq_record in SeqIO.parse('PKD2.gb','genbank'): # La única línea que cambió
    print(seq_record.id)           # id
    print(seq_record.name)         # nombre
    print(seq_record.description)  # descripción
    print(repr(seq_record.seq))    # secuencia
    print(len(seq_record))         # largo de secuencia
    
    print(seq_record.annotations['source']) # origen de la muestra
    print(f' Número de features: {len(seq_record.features)}')

Los archivos GenBank poseen mucha información, incluyendo, además de la secuencia misma, la anotación de los features y la bibliografía asociada.

Es posible obtener los features de manera independiente recorriendo el miembro **.features**

In [ ]:
# SeqFeature
for i in seq_record.features:
    if i.type == 'CDS':   #probar con 'exon'
        print(i)
        print('----------------------')
        print(i.location)
        print('======================')
    

<img src="images/entrez_logo.png" alt="Logo entrez" style="width: 200px;"/>

## Accediendo a NCBI vía Entrez

**Entrez** es un sistema de base de datos y de consulta a cerca de 20 bases de datos de NCBI a través de Internet.

Biopython provee una forma de generar consultas a Entrez, para descargar información desde NCBI en diversos formatos.

A continuación se muestra un ejemplo en el cual se consulta a la base de datos de nucleótidos sobre el registro de ID 205277388 (como argumento también se puede usar el Accession number), y se solicita obtener como respuesta un archivo FASTA en formato texto.

> **¿Qué son los "handlers"?**<br>
> Al manejar grandes volúmenes de datos es poco práctico cargarlos en memoria, pues se corre el riesgo de agotar los resursos del sistema. Una alternativa a esto es recorrer los archivos a medida que se leen, retrasando la lectura de una parte del archivo hasta que sea necesario, y descartando la información ya consumida. Un handler es un objeto que pemtite hacer esto.


In [ ]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "micorreo@uach.cl"

with Entrez.efetch(db="nucleotide", rettype="fasta", retmode="text", id="205277388") as handle: #ID: 205277388, Accession number: NG_008604.1
    seq_record = SeqIO.read(handle, "fasta")
    
    print(f'{seq_record.id} with {len(seq_record.features)} features')
    print(f'Primeras 500 letras de la secuencia: {seq_record.seq[0:500]}')

En el ejemplo anterior, no se muestran features porque el archivo solicitado (FASTA) no contiene información sobre features. Si repetimos la búsqueda, pero esta vez solicitamos el retorno de un archivo GenBank, se obtiene lo siguiente:

In [ ]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "micorreo@uach.cl"

with Entrez.efetch(db="nucleotide", rettype="gb", retmode="text", id="205277388") as handle:
    seq_record = SeqIO.read(handle, "gb")

    print(f'{seq_record.id} with {len(seq_record.features)} features:')
    print(f'\n------------------------ .name: {seq_record.name}')
    print(f'\n------------------------ .description: {seq_record.description}')
    print(f'\n------------------------ .letter_annotations: {seq_record.letter_annotations}')
    print(f'\n------------------------ .annotations: {seq_record.annotations}')
    print(f'\n------------------------ .dbxrefs: {seq_record.dbxrefs}')
    print('\n------------------------ .features:')
    for a in seq_record.features:
        print(a)


## BLAST

BLAST es una de las herramientas más populares entre los investigadores de genética. Permite enviar una secuencia y determinar en algunos segundos o minutos un listado de posibles alineamientos contra todos los organismos almacenados en las bases de datos de NCBI, ordenados de acuerdo a su probabilidad.

Biopython permite BLASTear una secuencia ya sea utilizando el servicio en Internet, o bien localmente. Note que la segunda alternativa requiere mantener localmente actualizados los genomas de referencia e índices de todas los organismos contra los cuales se desee contrastar la secuencia query.


In [ ]:
from Bio.Blast import NCBIWWW
help(NCBIWWW.qblast) # para obtener ayuda

El método para realizar las consultas es **qblast**, el cual se invoca con 3 argumentos:
1. El algoritmo que se invoca: **blastn** (nucleótidos), blastp, blastx, tblast o tblastx.
2. La base de datos que se consulta: **nt**, genome, protein, SNP, etc.
3. La secuencia query: ya sea como **secuencia**, un archivo FASTA o un *GI number* (Identificador de secuencia, reemplazado por los *Accession numbers* en 2016)

El resultado de una consulta vía qblast es un handler a los datos. Dado que los handlers sólo permiten leer los datos una vez, es buena idea almacenar la respuesta en un archivo. Esto se puede hacer en varios formatos, pero se recomienda XML por ser el más estable. El siguiente ejemplo ilustra este proceso. Dependiendo de la complejidad de la búsqueda y de la carga del servidor, la ejecución puede tardar algunos minutos.


In [ ]:
from Bio.Blast import NCBIWWW
from Bio.Seq import Seq
secuencia = Seq('agagtgagactctgtctcaaacaaaaaaaaaaacagagacagaaaaaaagaaagaaaatatatggatgtatatcatataaaaatataaataagggaggccaagtgcagtggcatgcctgtaatcccagcactttgggaggctgaagcaggaggatcacttgaggccgagaattcgagaccagcctgggcaacgtattgagacctcatctctgcaaaaaatcaaaaaatgaggcggaaggatggcttgagcccaggagatcaagccttcagtgagctgtgatcgtaccactacactcca')

result_handle = NCBIWWW.qblast("blastn", "nt", secuencia)

with open("resultado_blast.xml", "w") as out_handle:
    out_handle.write(result_handle.read())
result_handle.close()
print('***archivo almacenado***')

El archivo XML generado se puede explorar a simple vista o con ayuda de algún visor de XML (un visor online es https://xml.onlineviewer.net/), y contiene todos los alineamientos que BLAST encontró para la secuencia enviada. Entre los conceptos relevantes se encuentran los siguientes:

* **Hit**: representa una secuencia en la BD en donde la secuencia query hizo match. Un hit es títpicamente una secuencia almacenada en NCBI con la que la secuencia query tuvo similitud.

* **HSP**: (**H**igh-scoring **S**egment **P**air) representa una instancia de match significativo entre la secuencia de query y el hit. Un HSP es una *parte* de la secuencia "Hit" en la que se logra un alineamiento con pocos o ningún gap (lo que sería una diagonal en la matriz de Needleman-Wunsh o Smith-Waterman).

Los niveles anidados del XML, representan diferentes objetos de Biopython, que se estructuran de la siguiente manera:

- **BlastOutput** retorno de la consulta a BLAST
	- **BlastOutput_iterations** Listado invocaciones a BLAST (puede generarse un XML de múltiples consultas)
		- **Iteration** Cada iteración corresponde a una consulta a BLAST con una secuencia query
			- **Iteration_hits** Listado de **hits**. Puede haber uno o varios hits
                - **Hit** Cada **hit** encontrado.
					- **Hit_hsps** Listado de **HSP**s (**H**igh-scoring **S**egment **P**air) 
						- **Hsp** Cada **HSP**, incluyendo datos como la posición dentro del hit, el largo del alineamiento, el score del alineamiento y el alineamiento mismo

Este archivo puede ser grande. A continuación se despliegan las 60 primeras líneas del XML obtenido anteriormente


In [ ]:
!head -n 60 resultado_blast.xml

### Parseando la respuesta de BLAST

Una vez almacenado el archivo, los registros pueden recorrerse utilizando la librería NCBIXML. Existen básicamente dos métodos para parsear resultados: **read** que se utiliza (como en nuestro ejemplo) cuando el XML contiene los resultados de solo una consulta a BLAST (i.e., una iteración), y **parse**, cuando contiene más de una.

Si se utiliza **read**, se obtienen los datos de la única iteración del XML y del cual se pueden obtener los distintos Hits y HSPs.

Si se usa **parse**, se obtiene un handler, del cual se van obteniendo las iteraciones una a una, los cuales puede procesarse de la misma forma anterior

(Note que los nombres de los campos en el XML no necesariamente coinciden con los del objeto de Biopython)


In [ ]:
resultados = open("resultado_blast.xml")

from Bio.Blast import NCBIXML

# alternativa 1: solo una consulta
blast_record = NCBIXML.read(resultados)

# alternativa 2: más de una consulta
#blast_records = NCBIXML.parse(result_handle) # retorna handler
#blast_record = next(blast_records) # del handler obtiene el primer registro

# variables para limitar visualización
hits_mostrados = 0
hsps_mostrados = 0
max_hits_a_mostrar = 2
max_hsps_a_mostrar = 3

for alineamiento in blast_record.alignments:  # alignments se refiere a los hits
    for hsp in alineamiento.hsps: # recore HSPs
        print(f'=== Alineamiento  (Hit:{hits_mostrados} HSP:{hsps_mostrados}) ===================')
        print(f' Secuencia          : {alineamiento.title}') # title concatena tags hit_id y hit_def
        print(f' Largo hit          : {alineamiento.length}') # se refiere al largo del gentag hit_len
        print(f' Largo alineamiento : {hsp.align_length}') # largo del alineamiento (c.f. help(hsp))
        print(f' e value            : {hsp.expect}') # se refiere al tag Hsp_evalue

        # lo siguiente es solo para limitar la cantidad de registros mostrados
        hsps_mostrados += 1
        if hsps_mostrados >= max_hsps_a_mostrar:
            hsps_mostrados = 0
            break
    hits_mostrados += 1
    if hits_mostrados >= max_hits_a_mostrar:
        break

resultados.close()


## Tarea 4

Cree un programa ejecutable desde la línea de comandos, que al ser invocado reciba como argumento una secuencia query. Esta secuencia deberá ser BLASTeada online. De los resultados obtenidos (correspondientes a alineamientos), el programa deberá considerar el primer HSP del primer Hit y extraer el identificador del gen en el cual se encuentra, para posteriormente descargar información sobre este gen y mostrar la siguiente información:

- Especie a la que pertenece
- Gen al cual pertenece, indicando:
    - cromosoma en el que se encuentra
    - Ubicación y hebra (+/-)
    - número de exones que posee el gen

Ejemplo:

> $ ./Blastea_e_informa.py CTAGCTAGCTAGCTAGTCATGCATGCTAGCTACTCGATCG<br>
> BLASTeando secuencia...<br>
> Obteniendo información del gen...<br>
>   Especie: Homo sapiens<br>
>   Gen identificado: DNAJB11<br>
>   Ubicación: 3q27.3 +<br>
>   Exones en el gen: 11